In [ ]:
!pip install ccxt

     |████████████████████████████████| 3.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 60.8 MB/s 
     |████████████████████████████████| 1.1 MB 35.5 MB/s 
     |████████████████████████████████| 4.0 MB 33.7 MB/s 
     |████████████████████████████████| 271 kB 59.5 MB/s 
     |████████████████████████████████| 94 kB 658 kB/s 
     |████████████████████████████████| 291 kB 61.7 MB/s 
     |████████████████████████████████| 144 kB 61.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is

In [ ]:
import numpy as np
import pandas as pd
import ccxt
import time
import datetime

In [ ]:
api_key = ""
secret = ""

In [ ]:
class execute:
  def __init__ (self, api_key, secret, strategy='rmr', leverage=1, robust=symbols):
    self.exchange = ccxt.binance(config={
      'apiKey': api_key,
      'secret': secret,
      'enableRateLimit': True,
      'options': {
          'defaultType': 'future'}})
    self.robust = robust
    self.total_balance = self.get_total_balance()
    self.historic_prices =  self.get_historic_prices()
    if strategy == 'rmr':
      self.signal = self.relative_mean_reversion(self.historic_prices, 1, 5)
    self.weight = self.equal_weight(self.signal)
    self.signal_with_position = self.get_positions(self.signal)
    self.order_sheet = self.get_order_sheet(self.signal_with_position, self.total_balance * leverage, self.weight)
    self.trade(self.order_sheet, 3)
    self.check = self.get_bill(self.order_sheet)
    self.order_sheet2 = self.get_order_sheet2(self.check, self.total_balance * leverage, self.weight)
    self.trade(self.order_sheet2, 3)
    self.bill = self.get_bill(self.order_sheet2)
  
  def bool_converter(self, bool_var):
    if bool_var == True:
        result = 1
    elif bool_var == False:
        result = 0
    return result

  def get_historic_prices(self):
    df = pd.DataFrame(self.exchange.fetch_ohlcv('BTC/USDT', '4h', limit=4), columns=['datetime', 'open', 'high', 'low', 'BTC/USDT', 'volume'])
    df["datetime"] = pd.to_datetime(df["datetime"], unit='ms')
    df.set_index('datetime', inplace=True)
    date = df.index.tolist()
    historic_prices =  df['BTC/USDT']
    for symbol in self.exchange.markets.keys():
      if (symbol != 'BTC/USDT') and ('/USDT' in symbol) and symbol.split('/')[0] in self.robust:
        df_ = pd.DataFrame(self.exchange.fetch_ohlcv(symbol, '4h', limit=4), columns=['datetime', 'open', 'high', 'low', symbol, 'volume'], index=date)
        close = df_[symbol]
        historic_prices = pd.concat([historic_prices, close], axis=1)
    return historic_prices

  def relative_mean_reversion(self, prices, lookback, n_selection, long_only=False, short_only=False):
      returns = prices.pct_change(periods=lookback)[-1:]
      rank = returns.rank(axis=1, ascending=False)
      long_signal = (rank >= len(rank.columns) - n_selection + 1).applymap(self.bool_converter)
      short_signal = -(rank <= n_selection).applymap(self.bool_converter)
      if long_only == True:
        signal = long_signal
      elif short_only == True:
        signal = short_signal
      else:
        signal = long_signal + short_signal
      signal = signal.transpose()
      signal.columns = ['signal']
      return signal
  
  def get_positions(self, signal):
    position_info = self.exchange.fetch_balance()['info']['positions']
    signal['position'] = None
    signal['position_in_usdt'] = None
    for index in signal.index:
      symbol = index.replace('/','')
      for position in position_info:
        if position["symbol"] == symbol:
          signal['position'][index] = position['positionAmt']
          signal['position_in_usdt'][index] = position['notional']
    signal = signal.astype(float)
    return signal
  
  def get_rightnow_prices(self, signal):
    signal['price'] = None
    for index in signal.index:
      signal['price'][index] = self.exchange.fetch_ticker(index)['close']
    return signal

  def equal_weight(self, signal):
      total_signal = 1 / abs(signal).sum(axis=0)
      total_signal = total_signal.item()
      weight = pd.DataFrame(index=signal.index, columns=['weight']).fillna(value=1)
      weight = weight * total_signal
      signal = signal.astype(float)
      return weight

  def set_leverage(self,leverage, symbol):
    self.exchange.set_leverage(leverage, symbol)
  
  def get_total_balance(self):
    total = self.exchange.fetch_balance()['total']
    symbols = []
    amounts = []
    prices = []
    for symbol, amount in total.items():
      if symbol == 'USDT' or 'USDC':
        pass
      else:
        symbols.append(symbol+'/USDT')
        amounts.append(amount)
    for symbol in symbols:
      if symbol == 'USDT' or 'USDC':
        pass
      else:
        prices.append(self.exchange.fetch_ticker(symbol))
    symbols = np.array(symbols)
    amounts = np.array(amounts)
    prices = np.array(prices)
    balalnce_in_usdt = amounts * prices
    total_balance = balalnce_in_usdt.sum() + total['USDT']
    total_balance = float(total_balance)
    return total_balance
  
  def get_order_sheet(self, signal, balance, weight):
    signal['price'] = self.historic_prices[-1:].transpose()
    signal['additional_position_in_usdt'] = signal['signal'] * balance * weight['weight'] - signal['position'] * signal['price']
    signal['additional_position_in_amount'] = signal['additional_position_in_usdt'] / signal['price']
    signal = signal.astype(float)
    signal = signal.round(3)
    return signal

  def get_order_sheet2(self, signal, balance, weight):
    signal['additional_position_in_usdt'] = signal['signal'] * balance * weight['weight'] - signal['position'] * signal['price']
    signal['additional_position_in_amount'] = signal['additional_position_in_usdt'] / signal['price']
    signal = signal.astype(float)
    signal = signal.round(3)
    return signal

  def get_bill(self, order_sheet):
    order_sheet = order_sheet[order_sheet['signal'] != 0]
    position_info = self.exchange.fetch_balance()['info']['positions']
    for index in order_sheet.index:
      symbol = index.replace('/','')
      for position in position_info:
        if position["symbol"] == symbol:
          order_sheet['position'][index] = position['positionAmt']
          order_sheet['position_in_usdt'][index] = position['notional']
    order_sheet.astype(float)
    return order_sheet
  
  def trade(self, order_sheet, leverage=1):
    order_sheet = order_sheet[order_sheet['additional_position_in_amount'] != 0]
    for index in order_sheet.index:
      if order_sheet['additional_position_in_amount'][index] > 0:
        try:
          self.set_leverage(leverage, index)
          self.exchange.create_market_buy_order(index, order_sheet['additional_position_in_amount'][index])
        except: 
          pass
      if order_sheet['additional_position_in_amount'][index] < 0:
        try:
          self.set_leverage(leverage, index)
          self.exchange.create_market_sell_order(index, abs(order_sheet['additional_position_in_amount'][index]))
        except: 
          pass
    

In [ ]:
while True:
  now = datetime.datetime.now()
  if now.hour % 4 == 3 and 55 < now.minute < 57:
    execute = execute(api_key, secret)
    execute.bill.to_excel(f'{now.year}-0{now.month}-{now.day}-{now.hour+1}:00.xlsx')
  time.sleep(30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to

KeyboardInterrupt: ignored